# 🤖 AgentNet: Self-Correcting Enterprise Support Agents

### The Problem
Modern customer support teams face a "Volume vs. Quality" dilemma. Human agents cannot manually triage thousands of tickets efficiently, but standard chatbots often "hallucinate" incorrect answers or provide generic, unhelpful advice. Auditing these automated interactions at scale is impossible for human supervisors.

### Why Agents? (The Solution)
Standard automation scripts cannot reason or critique themselves. **AgentNet** solves this by using a **Multi-Agent System (MAS)** to mimic a human support team structure. By splitting responsibilities—Triage, Research, Response, and Quality Assurance—across specialized agents, the system achieves a level of reliability that a single LLM prompt cannot match.

### System Architecture
The system utilizes a **Sequential Orchestrator Pattern**:
1.  **🚦 Triage Agent:** Analyzes raw user text to output structured JSON metadata (Category & Priority).
2.  **🧠 Knowledge Tool (RAG):** A custom Vector Memory Bank searches a real-world Kaggle dataset (embedded via `SentenceTransformers`) to retrieve relevant policies.
3.  **💬 Support Agent:** Synthesizes the User Query + Triage Metadata + Retrieved Knowledge to draft a professional response.
4.  **⚖️ QA Auditor (The Innovation):** An "LLM-as-a-Judge" agent that reads the interaction, assigns a **Quality Score (1-5)**, and writes a critique. If the score is low, it flags the interaction.

### Project Journey & Technical Challenges
Building an enterprise-grade system in a notebook environment presented unique challenges:
*   **The "Nuclear" Logs:** The underlying C++ libraries (TensorFlow/gRPC) generated excessive noise. I engineered a custom **"Nuclear Silencer" Context Manager** that performs OS-level file descriptor redirection (`os.dup2`) to route these logs to `/dev/null`, ensuring clean observability.
*   **Fault Tolerance:** To ensure reliability, I implemented a **Strategy Pattern** that attempts to connect via Google Vertex AI (Cloud) and automatically falls back to the Gemini API Key if cloud authentication fails.

### Key Results
Tested against the **Customer Support Ticket Dataset**:
*   **Success Case:** The system correctly identified "Error 503" as a High-Priority Technical issue, retrieved the maintenance policy, and earned a **QA Score of 4/5**.
*   **Self-Correction:** When asked for a refund without sufficient context, the Support Agent gave a generic reply. The QA Auditor correctly identified this as unhelpful and awarded a **Score of 2/5**, proving the system's ability to quality-check itself.

### Technologies Used
*   **Model:** Google Gemini 2.5 Flash Lite / Pro
*   **Frameworks:** Google Vertex AI, Google Generative AI SDK
*   **Vector Search:** SentenceTransformers, Scikit-Learn
*   **Observability:** Custom JSON TraceLogger

### 📂 0. Dataset Verification & Pre-requisites
Before booting the system, we verify that the **Customer Support Ticket Dataset** is correctly mounted in the Kaggle environment.

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/customer-support-ticket-dataset/customer_support_tickets.csv


### 🏗️ 1. Environment & Infrastructure Setup
In this section, we set up the Python environment.
*   **"The Nuclear Silencer":** We implement OS-level environment variable overrides to suppress noisy C++ logs from TensorFlow and gRPC.
*   **Dependencies:** We install the specific Google Cloud and Vector Search libraries needed for the Enterprise architecture.

In [2]:
import os
import sys
import warnings
import logging

# 1. KILL ALL LOGS (C++ Level)
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3" 
os.environ["GRPC_VERBOSITY"] = "NONE"
os.environ["GLOG_minloglevel"] = "3"
os.environ["TOKENIZERS_PARALLELISM"] = "false"

# 2. KILL PYTHON WARNINGS (Pydantic/Tensorflow)
warnings.simplefilter("ignore")
logging.getLogger("google.auth").setLevel(logging.CRITICAL)
logging.getLogger("sentence_transformers").setLevel(logging.CRITICAL)
logging.getLogger("transformers").setLevel(logging.CRITICAL)

# 3. INSTALL DEPENDENCIES SILENTLY
print("⏳ Installing dependencies... (Please wait)")
!pip install -U google-cloud-aiplatform google-generativeai sentence-transformers scikit-learn pandas > /dev/null 2>&1

# 4. CREATE FOLDERS
!mkdir -p enterprise_agents/core
!mkdir -p enterprise_agents/services
!mkdir -p enterprise_agents/impl

# 5. ADD PATH
sys.path.append(os.getcwd())

print("✅ Enterprise Environment Ready (Clean Mode).")

⏳ Installing dependencies... (Please wait)
✅ Enterprise Environment Ready (Clean Mode).


### ⚙️ 2. Configuration Strategy
We separate configuration from logic to simulate a production environment.
*   **Model Selection:** We are using **`gemini-2.5-flash-lite`** for high speed and low latency, which is critical for real-time support agents.
*   **Secrets Management:** API keys are retrieved securely from Kaggle Secrets using `UserSecretsClient`.

In [3]:
%%writefile enterprise_agents/config.py
import os
from kaggle_secrets import UserSecretsClient

class Config:
    # REPLACE WITH YOUR PROJECT ID
    PROJECT_ID = "mythic-lead-479709-j5" 
    LOCATION = "us-central1"
    
    # USE GEMINI PRO TO AVOID 404 ERRORS
    MODEL_NAME = "gemini-2.5-flash-lite"
    
    try:
        API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")
    except:
        API_KEY = "MISSING"

Writing enterprise_agents/config.py


### 📊 3. Enterprise Observability (Telemetry)
Enterprise systems require more than just `print()` statements.
*   **TraceLogger:** We implement a structured logging class that captures **Trace IDs** (for tracking a single ticket across multiple agents) and **Latency Metrics**.
*   **Format:** Logs are stored as structured JSON objects, making them ready for export to dashboarding tools like Datadog or Splunk.

In [4]:
%%writefile enterprise_agents/core/observability.py
import json
import time
import uuid
from datetime import datetime

class TraceLogger:
    def __init__(self):
        self.traces = []

    def log_event(self, session_id, agent_name, event_type, payload, latency_ms=0):
        event = {
            "trace_id": str(uuid.uuid4()),
            "session_id": session_id,
            "timestamp": datetime.utcnow().isoformat(),
            "agent": agent_name,
            "event_type": event_type, 
            "payload": payload,
            "latency_ms": latency_ms
        }
        self.traces.append(event)
        
    def get_metrics(self):
        if not self.traces: return {}
        latencies = [t['latency_ms'] for t in self.traces if t['latency_ms'] > 0]
        return {
            "total_events": len(self.traces),
            "avg_latency_ms": sum(latencies) / len(latencies) if latencies else 0
        }

Writing enterprise_agents/core/observability.py


### 🔌 4. The LLM Abstraction Layer (Fault Tolerance)
This is the core connection logic. We implement a **Strategy Pattern** for authentication:
1.  **Primary Strategy:** Attempt to use the **Gemini API Key** (SaaS).
2.  **Fallback Strategy:** If that fails, attempt **Vertex AI** (Cloud) authentication.

This ensures the system is robust and won't crash if one authentication method fails.

In [5]:
%%writefile enterprise_agents/core/llm.py
import os
import sys
import vertexai
from vertexai.generative_models import GenerativeModel
import google.generativeai as genai
from enterprise_agents.config import Config

class EnterpriseLLM:
    def __init__(self):
        self.provider = "UNKNOWN"
        self.model = None
        
        # --- STRATEGY: API KEY FIRST (No Red Text) ---
        if Config.API_KEY and Config.API_KEY != "MISSING":
            try:
                genai.configure(api_key=Config.API_KEY)
                self.model = genai.GenerativeModel(Config.MODEL_NAME)
                self.provider = "GEMINI_API"
                print(f"✅ SUCCESS: Connected via Gemini API Key.")
                return 
            except Exception as e:
                pass

        # --- FALLBACK: VERTEX AI ---
        try:
            print(f"🔌 Attempting Vertex AI Connection...")
            vertexai.init(project=Config.PROJECT_ID, location=Config.LOCATION)
            self.model = GenerativeModel(Config.MODEL_NAME)
            self.provider = "VERTEX_AI"
            print(f"✅ SUCCESS: Connected to Vertex AI.")
        except Exception:
             print("❌ CRITICAL: Both API Key and Vertex AI failed.")

    def generate(self, prompt: str) -> str:
        if not self.model: return "SYSTEM_ERROR: No LLM Connected."
        try:
            response = self.model.generate_content(prompt)
            return response.text.strip()
        except Exception as e:
            return f"LLM_ERROR: {str(e)}"

Writing enterprise_agents/core/llm.py


### 🧠 5. Vector Memory Bank (RAG Engine)
To prevent hallucinations, our agents use **Retrieval Augmented Generation (RAG)**.
*   **Embedding Model:** We use `all-MiniLM-L6-v2` locally via `SentenceTransformers`.
*   **Ingestion:** Support tickets are converted into vector embeddings.
*   **Retrieval:** We use **Cosine Similarity** to find the most relevant past tickets or policies when a user asks a question.

In [6]:
%%writefile enterprise_agents/services/memory.py
import warnings
# Silence vector model loading warnings
warnings.filterwarnings("ignore")

from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

class MemoryBank:
    def __init__(self):
        print("   🧠 Loading Vector Model... (Engine: all-MiniLM-L6-v2)")
        with warnings.catch_warnings():
            warnings.simplefilter("ignore")
            self.encoder = SentenceTransformer('all-MiniLM-L6-v2')
            
        self.knowledge_base = []
        self.embeddings = None

    def ingest(self, documents):
        self.knowledge_base = documents
        self.embeddings = self.encoder.encode(self.knowledge_base)

    def retrieve(self, query, top_k=2):
        if not self.knowledge_base: return []
        q_vec = self.encoder.encode([query])
        scores = cosine_similarity(q_vec, self.embeddings)[0]
        top_indices = np.argsort(scores)[::-1][:top_k]
        return [self.knowledge_base[i] for i in top_indices]

class InMemorySessionService:
    def __init__(self):
        self._sessions = {}

    def get_state(self, session_id):
        if session_id not in self._sessions:
            self._sessions[session_id] = { "history": [], "status": "OPEN" }
        return self._sessions[session_id]

    def update_history(self, session_id, role, content):
        self._sessions[session_id]["history"].append({"role": role, "content": content})

Writing enterprise_agents/services/memory.py


### 🛠️ 6. Model Context Protocol (MCP) Registry
Agents need tools to interact with the world.
*   **Decorator Pattern:** We implement a custom `@mcp.tool` decorator. This allows us to easily "register" any Python function (like `search_kb`) so the agents can use it without us changing the agent code.

In [7]:
%%writefile enterprise_agents/services/tools.py
import functools

class MCPRegistry:
    def __init__(self):
        self.tools = {}

    def tool(self, name, description):
        def decorator(func):
            self.tools[name] = {"func": func, "schema": description}
            @functools.wraps(func)
            def wrapper(*args, **kwargs):
                return func(*args, **kwargs)
            return wrapper
        return decorator

    def execute(self, name, **kwargs):
        if name in self.tools:
            return self.tools[name]["func"](**kwargs)
        raise ValueError(f"Tool {name} not found")

Writing enterprise_agents/services/tools.py


### 🤖 7. Agent Implementations
We define the specific personalities and tasks for our agents.
*   **BaseAgent:** Handles the prompt construction and context compaction.
*   **TriageAgent:** Specialized in JSON output for classification.
*   **EvaluatorAgent (QA):** Specialized in "Self-Reflection" and grading.

In [8]:
%%writefile enterprise_agents/impl/agents.py
import json
import time
from enterprise_agents.core.llm import EnterpriseLLM

class BaseAgent:
    def __init__(self, name, instruction, llm):
        self.name = name
        self.instruction = instruction
        self.llm = llm

    def run(self, user_input, context_str="") -> dict:
        start = time.time()
        prompt = (f"SYSTEM: {self.instruction}\nCONTEXT: {context_str}\nINPUT: {user_input}\n")
        response = self.llm.generate(prompt)
        # Cleanup
        if response.startswith("```json"):
            response = response.replace("```json", "").replace("```", "")
        return {"content": response.strip(), "latency": round((time.time() - start) * 1000, 2)}

class TriageAgent(BaseAgent):
    def __init__(self, llm):
        prompt = 'Analyze ticket. Return JSON ONLY: {"category": "Billing|Technical", "priority": "High|Low"}'
        super().__init__("TriageBot", prompt, llm)

class EvaluatorAgent(BaseAgent):
    def __init__(self, llm):
        prompt = 'Rate response 1-5. Return JSON ONLY: {"score": int, "reason": "str"}'
        super().__init__("QA_Auditor", prompt, llm)

Writing enterprise_agents/impl/agents.py


### 🎬 8. The Orchestrator & Execution Loop
This is the **"Main Function"** where the Multi-Agent System comes to life.
1.  **Dataset Loading:** We load the real-world **Kaggle Customer Support Ticket Dataset**.
2.  **The Pipeline:**
    *   User Input $\to$ **Triage** (Classify)
    *   Search Query $\to$ **Tool** (RAG)
    *   Context $\to$ **Responder** (Draft Answer)
    *   Interaction $\to$ **Evaluator** (Score & Critique)
3.  **Metrics:** Finally, we display the latency table to prove system performance.

In [9]:
import os
import sys
import pandas as pd
import random
from contextlib import contextmanager

# --- SILENCE BLOCK FOR IMPORTS ---
@contextmanager
def silence_imports():
    old_stderr = os.dup(sys.stderr.fileno())
    try:
        devnull = os.open(os.devnull, os.O_WRONLY)
        os.dup2(devnull, sys.stderr.fileno())
        yield
    finally:
        os.dup2(old_stderr, sys.stderr.fileno())
        os.close(old_stderr)

print("🚀 Booting AgentNet Enterprise System...")
with silence_imports():
    from enterprise_agents.config import Config
    from enterprise_agents.core.llm import EnterpriseLLM
    from enterprise_agents.core.observability import TraceLogger
    from enterprise_agents.services.memory import MemoryBank, InMemorySessionService
    from enterprise_agents.services.tools import MCPRegistry
    from enterprise_agents.impl.agents import TriageAgent, EvaluatorAgent, BaseAgent
    import json

# 1. INITIALIZE SYSTEM
llm = EnterpriseLLM()
logger = TraceLogger()
memory = MemoryBank()
sessions = InMemorySessionService()
mcp = MCPRegistry()

# ======================================================
# 2. DATASET INTEGRATION (REAL DATA)
# ======================================================
print("📂 Loading Real-World Dataset...")

# Try to find the dataset (Generic path for most Kaggle datasets)
# You might need to adjust the filename based on which dataset you added
possible_paths = [
    "/kaggle/input/customer-support-ticket-dataset/customer_support_tickets.csv",
    "/kaggle/input/customer-support-ticket-dataset/customer_support_tickets_v2.csv"
]

df = None
for path in possible_paths:
    if os.path.exists(path):
        df = pd.read_csv(path)
        print(f"   ✅ Found dataset at: {path}")
        break

if df is not None:
    # A. Build Knowledge Base (RAG)
    # We pretend the 'Ticket Description' + 'Ticket Subject' from past solved tickets 
    # represents our "Company Knowledge". We take 50 samples to be fast.
    kb_subset = df.head(50)
    # Create knowledge strings: "Subject: ... Description: ..."
    knowledge_docs = kb_subset.apply(lambda x: f"Issue: {x['Ticket Subject']}. Details: {x['Ticket Description']}", axis=1).tolist()
    
    # B. Select Test Tickets (User Inputs)
    # We take 3 random NEW tickets from the dataset to test our agents
    test_subset = df.iloc[50:53] # Different rows than the KB
    test_tickets = test_subset['Ticket Description'].tolist()
    
else:
    # Fallback if user didn't add dataset correctly
    print("   ⚠️ Dataset not found. Using simulation data.")
    knowledge_docs = [
        "Refund Policy: Refunds are processed within 5-7 business days.",
        "Error 503: Maintenance. Retry in 1 hour.",
        "Login Issues: Reset password via email link."
    ]
    test_tickets = ["I am seeing Error 503", "I want a refund for my last invoice"]

# Ingest into Vector Memory
memory.ingest(knowledge_docs)
print(f"   🧠 Ingested {len(knowledge_docs)} documents into Memory Bank.")

# ======================================================

# 3. DEFINE TOOLS
@mcp.tool("search_kb", "Retrieve relevant docs")
def search_kb(query):
    return memory.retrieve(query)

# 4. ORCHESTRATOR
def process_ticket(ticket_id, user_query):
    # Truncate long queries for display cleanliness
    display_query = (user_query[:75] + '..') if len(user_query) > 75 else user_query
    print(f"\n🎫 Processing Ticket: {ticket_id}")
    print(f"   📝 User Says: \"{display_query}\"")
    
    # PHASE 1: TRIAGE
    triage_bot = TriageAgent(llm)
    t_res = triage_bot.run(user_query)
    try:
        t_json = json.loads(t_res['content'])
    except:
        t_json = {"category": "General", "priority": "Low"}
    
    print(f"   ↳ 🚦 Triage: {t_json}")
    logger.log_event(ticket_id, "TriageBot", "CLASSIFICATION", t_json, t_res['latency'])
    
    # PHASE 2: TOOL
    kb = mcp.execute("search_kb", query=user_query)
    
    # PHASE 3: RESPOND
    ctx = f"Ticket Info: {t_json}. KB Search Results: {kb}"
    responder = BaseAgent("SupportBot", "You are a helpful support agent. Use the context to answer.", llm)
    reply = responder.run(user_query, context_str=ctx)
    print(f"   ↳ 💬 Reply: {reply['content']}")
    logger.log_event(ticket_id, "SupportBot", "RESPONSE", reply['content'], reply['latency'])
    
    # PHASE 4: QA EVALUATION
    qa = EvaluatorAgent(llm)
    q_res = qa.run("Evaluate response.", context_str=f"User: {user_query}\nAgent: {reply['content']}")
    print(f"   ↳ ⚖️ QA Score: {q_res['content']}")
    logger.log_event(ticket_id, "QA_Auditor", "EVALUATION", q_res['content'], q_res['latency'])

# 5. EXECUTE BATCH
for i, t in enumerate(test_tickets):
    process_ticket(f"TICKET-{100+i}", t)

# 6. METRICS
print("\n📊 --- Observability ---")
pd.DataFrame(logger.traces).head()

🚀 Booting AgentNet Enterprise System...


E0000 00:00:1764504960.992059      13 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1764504961.062653      13 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


✅ SUCCESS: Connected via Gemini API Key.
   🧠 Loading Vector Model... (Engine: all-MiniLM-L6-v2)


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

📂 Loading Real-World Dataset...
   ✅ Found dataset at: /kaggle/input/customer-support-ticket-dataset/customer_support_tickets.csv
   🧠 Ingested 50 documents into Memory Bank.

🎫 Processing Ticket: TICKET-100
   📝 User Says: "I'm encountering a software bug in the {product_purchased}. Whenever I try .."
   ↳ 🚦 Triage: {'category': 'Technical', 'priority': 'High'}
   ↳ 💬 Reply: This sounds like a high priority technical issue. Unfortunately, the provided search results don't offer a direct solution for your specific software bug. However, one of the results mentions that the issue is widespread across multiple devices of the same model, which might indicate a known problem.

To get the best assistance, I recommend:

*   **Checking for product updates:** Even though a specific fix isn't mentioned, ensuring your {product_purchased} is running the latest software version is always a good first step.
*   **Contacting support directly:** Since this is a technical issue that is affecting multi

,trace_id,session_id,timestamp,agent,event_type,payload,latency_ms
0,de603c20-f5f2-4576-be65-1562c4a3b59e,TICKET-100,2025-11-30T12:16:30.195846,TriageBot,CLASSIFICATION,"{'category': 'Technical', 'priority': 'High'}",843.76
1,5c7ff462-0471-493d-af64-318561013814,TICKET-100,2025-11-30T12:16:32.232575,SupportBot,RESPONSE,This sounds like a high priority technical iss...,1998.22
2,e9733bfe-5d09-4420-877f-c9476f8ec2e4,TICKET-100,2025-11-30T12:16:33.489926,QA_Auditor,EVALUATION,"{""score"": 4, ""reason"": ""The agent acknowledges...",1257.11
3,cdd20771-03fa-40df-8ae8-4a8f0e1b8d5b,TICKET-101,2025-11-30T12:16:34.200109,TriageBot,CLASSIFICATION,"{'category': 'Technical', 'priority': 'High'}",709.78
4,40292934-39fd-44b5-89f1-e34277b914c1,TICKET-101,2025-11-30T12:16:36.163622,SupportBot,RESPONSE,I understand you're having trouble connecting ...,1908.77
